In [1]:
import pandas as pd
import statsmodels.api as sm
import math
import numpy as np

In [2]:
reds = pd.read_csv("RED_all_faults.csv")
greys = pd.read_csv("GREY_all_faults.csv")

In [ ]:
print(greys.shape)
nan_count = greys.isna().sum()
print(nan_count)

In [ ]:
print(reds.shape)
nan_count = reds.isna().sum()
print(nan_count)

In [3]:
# Drop gross_combined_weight
reds = reds.drop(columns=['gross_combined_weight'])
greys = greys.drop(columns=['gross_combined_weight'])

In [ ]:
reds['red_fault'] = 1
greys['red_fault'] = 0

In [4]:
faults = pd.concat([reds, greys])

In [ ]:
print(faults.shape)
nan_count = faults.isna().sum()
print(nan_count)

(6665381, 53)
component_id                  0
fault_alarm_id                0
vehicle_id                    0
message_date_time             0
longitude                     0
latitude                      0
city_name                 25438
state_cd                  37265
fault_count                   0
component_type                0
fmi_id                        0
ecu_type                      0
fault_active                  0
position_timestamp            0
create_date                   0
heading                       0
elevation                     0
date                          0
odometer_reading             18
engine_status                 0
idle_engine_hours             0
total_fuel                48262
engine_hours             314462
rank                          0
vehicle_id.1                  0
vehicle_type                  0
vehicle_spec_id               0
vehicle_model_year            0
vehicle_model                 0
vehicle_make                  0
veh_version               

In [ ]:
display(faults)

,component_id,fault_alarm_id,vehicle_id,message_date_time,longitude,latitude,city_name,state_cd,fault_count,component_type,...,final_assembly_dt,factory_invoice_dt,engine_type,engine_make,dispatch_dt,create_dt,country_of_operation,chassis_series,chassis_number,red_fault


In [ ]:
faults = faults.drop(columns=['inactive_cd'])
faults = faults.drop(columns=['dispatch_dt'])
faults = faults.drop(columns=['create_date'])
faults = faults.drop(columns=['date'])
faults = faults.drop(columns=['rank'])
faults = faults.drop(columns=['vehicle_id.1'])
faults = faults.drop(columns=['vehicle_spec_id'])
faults = faults.drop(columns=['retail_dt'])
faults = faults.drop(columns=['mq_message_put_dt'])
faults = faults.drop(columns=['main_assembly_dt'])
faults = faults.drop(columns=['final_assembly_dt'])
faults = faults.drop(columns=['factory_invoice_dt'])
faults = faults.drop(columns=['create_dt'])
faults = faults.drop(columns=['component_id'])
faults = faults.drop(columns=['vehicle_id'])
faults = faults.drop(columns=['fmi_id'])
faults = faults.drop(columns=['position_timestamp'])

In [ ]:
faults = faults.dropna()

In [ ]:
# Transforms the idle_engine_hours to hours
faults['idle_engine_hours'] = faults['idle_engine_hours'] / 3600

In [ ]:
# Calculates the active_engine_hours
faults['active_engine_hours'] = faults['engine_hours'] - faults['idle_engine_hours']
print(faults['active_engine_hours'])

0           549.148902
1           534.003028
2           532.453403
3           530.453916
4           532.453403
              ...     
6593672    7055.311277
6593673    7055.311277
6593674    7055.311277
6593675    2154.108958
6593676    4990.853263
Name: active_engine_hours, Length: 6267501, dtype: float64


In [ ]:
# Calculate the vehicle service age in days
faults['vehicle_service_age'] = pd.to_datetime(faults['message_date_time']) - pd.to_datetime(faults['in_service_dt'])
faults['vehicle_service_age'] = faults['vehicle_service_age'].dt.days
print(faults['vehicle_service_age'])

0          535
1          446
2          446
3          432
4          446
          ... 
6593672    618
6593673    618
6593674    618
6593675    780
6593676    658
Name: vehicle_service_age, Length: 6267501, dtype: int32


In [ ]:
# Turns vehicle make into a boolean
volvor = lambda x: 1 if x == "VOLVO" else 0
faults['volvo'] = faults['vehicle_make'].apply(volvor)
print(faults['volvo'].mean())

0.6712343564045702


In [ ]:
with pd.option_context('display.max_columns', None):  # more options can be specified also
    display(faults)

,fault_alarm_id,message_date_time,longitude,latitude,city_name,state_cd,fault_count,component_type,ecu_type,fault_active,heading,elevation,odometer_reading,engine_status,idle_engine_hours,total_fuel,engine_hours,vehicle_type,vehicle_model_year,vehicle_model,vehicle_make,veh_version,tgw_serial,product_type,notif_type,model_family,in_service_dt,in_process,imsi,gross_wt_rating,engine_type,engine_make,country_of_operation,chassis_series,chassis_number,red_fault,active_engine_hours,vehicle_service_age,volvo
0,2576200448,2022-11-14 19:40:35,-87.559426,33.197592,Tuscaloosa,AL,1,SPN,3,Y,184.0,180.0,5813.182,0,0.102097,1359.561467,549.251,D,2021,GR64B,MACK,78,20387781.0,CML,UPDATED,GR,2021-05-28 00:00:00,N,3.104103e+14,31207.0,US20,VOLVO,USA,M335,23421,1,549.148902,535,0
1,2419946708,2022-08-17 16:15:19,-86.806965,33.610952,Fultondale,AL,1,SPN,61,Y,179.0,568.0,5812.126,0,0.097973,1347.673725,534.101,D,2021,GR64B,MACK,78,20387781.0,CML,UPDATED,GR,2021-05-28 00:00:00,N,3.104103e+14,31207.0,US20,VOLVO,USA,M335,23421,1,534.003028,446,0
2,2419865080,2022-08-17 15:37:59,-86.806965,33.610952,Fultondale,AL,1,SPN,61,I,179.0,568.0,5807.143,1,0.097598,1345.824521,532.551,D,2021,GR64B,MACK,78,20387781.0,CML,UPDATED,GR,2021-05-28 00:00:00,N,3.104103e+14,31207.0,US20,VOLVO,USA,M335,23421,1,532.453403,446,0
3,2397017565,2022-08-03 19:44:25,-86.571504,34.854532,Meridianville,AL,1,SPN,61,Y,199.0,751.0,5806.291,0,0.097084,1342.918628,530.551,D,2021,GR64B,MACK,78,20387781.0,CML,UPDATED,GR,2021-05-28 00:00:00,N,3.104103e+14,31207.0,US20,VOLVO,USA,M335,23421,1,530.453916,432,0
4,2420154551,2022-08-17 17:51:09,-86.806965,33.610952,Fultondale,AL,1,SPN,61,N,179.0,568.0,5807.143,1,0.097598,1345.824521,532.551,D,2021,GR64B,MACK,78,20387781.0,CML,UPDATED,GR,2021-05-28 00:00:00,N,3.104103e+14,31207.0,US20,VOLVO,USA,M335,23421,1,532.453403,446,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6593672,2679841006,2022-12-03 02:23:36,-113.542537,53.339924,Leduc County,AB,10,SPN,3,I,0.0,2323.0,288136.520,1,0.598723,39977.288769,7055.910,D,2022,VNL64T N,VOLVO,94,21027767.0,VNL6,UPDATED,VN,2021-03-25 00:00:00,N,3.104103e+14,24131.0,US21,VOLVO,CAN,NR,287473,0,7055.311277,618,1
6593673,2679841065,2022-12-03 02:25:56,-113.542636,53.339950,Leduc County,AB,12,SPN,3,Y,0.0,2359.0,288136.520,1,0.598723,39977.288769,7055.910,D,2022,VNL64T N,VOLVO,94,21027767.0,VNL6,UPDATED,VN,2021-03-25 00:00:00,N,3.104103e+14,24131.0,US21,VOLVO,CAN,NR,287473,0,7055.311277,618,1
6593674,2679841088,2022-12-03 02:41:33,-113.542581,53.339986,Leduc County,AB,13,SPN,3,Y,0.0,2369.0,288136.520,1,0.598723,39977.288769,7055.910,D,2022,VNL64T N,VOLVO,94,21027767.0,VNL6,UPDATED,VN,2021-03-25 00:00:00,N,3.104103e+14,24131.0,US21,VOLVO,CAN,NR,287473,0,7055.311277,618,1
6593675,2687062844,2022-07-08 18:38:25,-118.809534,55.076558,Grovedale,AB,0,PSID,216,N,234.0,1900.0,33070.506,1,0.194042,7192.480384,2154.303,D,2020,AN64T,MACK,130,19095351.0,CMM,UPDATED,AN,2020-05-19 00:00:00,N,3.104102e+14,24200.0,US19,VOLVO,CAN,M742,13591,0,2154.108958,780,0


In [ ]:
faults = pd.get_dummies(faults, columns=['state_cd', 'component_type', 'fault_active', 'vehicle_type', 'vehicle_model_year', 'vehicle_model', 'veh_version', 'product_type', 'notif_type', 'model_family', 'in_process', 'engine_type', 'engine_make', 'country_of_operation', 'chassis_series'], drop_first=True)

: 

: 

In [6]:
faults.value_counts('state_cd')

state_cd
TX    579384
IL    433384
GA    370186
CA    345481
PA    293990
       ...  
QR        96
DF        95
MX        38
NT        29
BS         4
Name: count, Length: 87, dtype: int64